# Modules, fichiers, expressions régulières (correction)

Correction.

In [1]:
%matplotlib inline

### Exercice 1 : Excel $\rightarrow$ Python $\rightarrow$ Excel
    
On récupère le fichier [seance4_excel.csv](https://github.com/sdpython/teachpyx/raw/td/_data/seance4_excel.csv) :

- le lire sous python
- créer une matrice carrée 3x3 où chaque valeur est dans sa case (X,Y),
- enregistrer le résultat sous format texte,
- le récupérer sous Excel.    

On utilise le module `urllib` pour télécharger automatiquement le fichier :

In [3]:
from urllib.request import urlopen

url = "https://github.com/sdpython/teachpyx/raw/td/_data/seance4_excel.csv"
filename = "seance4_excel.csv"
with urlopen(url) as u:
    with open(filename, "w") as f:
        f.write(u.read())

In [ ]:
with open("seance4_excel.csv", "r") as f:
    mat = [row.strip(" \n").split("\t") for row in f.readlines()]

mat = mat[1:]
res = [[None] * 3 for i in range(5)]
for i, j, v in mat:
    res[int(j) - 1][int(i) - 1] = float(v)

with open("seance4_excel_mat.csv", "w") as f:
    f.write("\n".join(["\t".join([str(x) for x in row]) for row in res]))

Il est très rare d'écrire ce genre de code. En règle générale, on se sert de modules déjà existant comme [pandas](http://pandas.pydata.org/), [xlrd](http://www.python-excel.org/) et  [openpyxl](http://pythonhosted.org/openpyxl/). Cela évite la conversion au format texte :

In [ ]:
import pandas

df = pandas.read_excel("seance4_excel.xlsx", sheet_name="Feuil1", engine="openpyxl")
mat = df.pivot("X", "Y", "value")
mat.to_excel("seance4_excel_mat.xlsx")
mat

C'est un peu plus rapide.

<h3 id="exo2">Exercice 2 : trouver un module (1)</h3>

Le module [random](https://docs.python.org/3.4/library/random.html) est celui qu'on cherche.

In [ ]:
import random

alea = [random.random() for i in range(10)]
print(alea)
random.shuffle(alea)
print(alea)

### Exercice 3 : trouver un module (2)

Le module [datetime](https://docs.python.org/3.4/library/datetime.html) permet de faire des opérations sur les dates.

In [ ]:
from datetime import datetime

date1 = datetime(2013, 9, 9)
date0 = datetime(2013, 8, 1)
print(date1 - date0)
birth = datetime(1975, 8, 11)
print(birth.weekday())  # lundi

### Exercice 4 : son propre module

On effectue le remplacement ``if __name__ == "__main__":``  par ``if True :`` :

In [ ]:
# fichier monmodule2.py
import math


def fonction_cos_sequence(seq):
    return [math.cos(x) for x in seq]


if __name__ == "__main__":
    # et une petite astuce quand on travaille sous notebook
    code = """
        # -*- coding: utf-8 -*-
        import math
        def fonction_cos_sequence(seq) :
            return [ math.cos(x) for x in seq ]        
        if True :
            print ("Ce message n'apparaît que si ce programme est le point d'entrée.")
        """.replace(
        "        ", ""
    )
    with open("monmodule3.py", "w", encoding="utf8") as f:
        f.write(code)

In [ ]:
import monmodule3

print(monmodule3.fonction_cos_sequence([1, 2, 3]))

Le message ``ce message n'apparaît que ce programme est le point d'entrée`` apparaît maintenant alors qu'il n'apparaissait pas avec la version de l'énoncé. Comme il apparaît après ``*``, cela montre que cette ligne est exécutée si le module est importé.

In [ ]:
import monmodule3

Si on importe le module une seconde fois, le message n'apparaît plus : le langage Python a détecté que le module avait déjà été importé. Il ne le fait pas une seconde fois.

### Exercice 5 : chercher un motif dans un texte

L'expression régulière est ``je .{1,60}``. Le symbol ``.`` signifie n'importe quel caractère. Suivi de ``{1,60}`` veut dire n'importe quel caractère répété entre 1 et 60 fois.

In [ ]:
from teachpyx.tools.data_helper import download_and_unzip

url = "https://github.com/sdpython/teachpyx/raw/td/_data/voeux.zip"
discours = download_and_unzip(url)
discours

On affiche le contenu du premier fichier.

In [ ]:
with open("VOEUX01.txt", "r") as f:
    print(f.read())

In [ ]:
exp = re.compile("je .{1,60}", re.IGNORECASE)
for fichier in discours:
    print("----", fichier)
    try:
        with open(fichier, "r") as f:
            text = f.read()
    except:
        with open(fichier, "r", encoding="latin-1") as f:
            text = f.read()
    je = exp.findall(text)
    for t in je:
        print(t)

### Exercice 6 : chercher un autre motif dans un texte

Pour les mots _securite_ ou _insecurite_, on construit l'expression :

In [ ]:
exp = re.compile("(.{1,15}(in)?sécurité.{1,50})", re.IGNORECASE)
for fichier in discours:
    print("----", fichier)
    try:
        with open(fichier, "r") as f:
            text = f.read()
    except:
        with open(fichier, "r", encoding="latin-1") as f:
            text = f.read()
    je = exp.findall(text)
    for t in je:
        print(t)

## Exercice 7 : recherche les urls dans une page wikipédia

On pourra prendre comme exemple la page du programme [Python](https://fr.wikipedia.org/wiki/Python_(langage)). La première partie consiste à récupérer le contenu d'une page HTML.

In [ ]:
from urllib.request import urlopen

url = "https://fr.wikipedia.org/wiki/Python_(langage)"
with urlopen(url) as u:
    content = u.read()
content[:300]

Les données récupérées sont au format binaire d'où le préfixe ``b''``. Pour éviter de télécharger les données à chaque fois, on sauve le contenu sur disque pour le récupérer la prochaine fois.

In [ ]:
with open("page.html", "wb") as f:
    f.write(content)

Et on le recharge.

In [ ]:
with open("page.html", "rb") as f:
    page = f.read()
page[:300]

Les données sont sous forme d'octets, il faut d'abord les convertir sous forme de caractères. il y a plus de caractères que d'octets disponibles (256), c'est cela qu'il faut une sorte de code pour passer de l'un à l'autre : dans le cas d'internet, le plus utilisé est l'[encoding utf-8](https://fr.wikipedia.org/wiki/UTF-8).

In [ ]:
page_str = page.decode("utf-8")
page_str[:300]

On recherche maintenant les urls commençant par http...

In [ ]:
import re

reg = re.compile('href=\\"(http.*?)\\"')
urls = reg.findall(page_str)
urls[:10]

## Exercice 8 : construire un texte à motif

A l'inverse des expressions régulières, des modules comme [Mako](http://www.makotemplates.org/) ou [Jinja2](http://jinja.pocoo.org/docs/dev/) permettent de construire simplement des documents qui suivent des règles. Ces outils sont très utilisés pour la construction de page web. On appelle cela faire du [templating](https://wiki.python.org/moin/Templating). Créer une page web qui affiche à l'aide d'un des modules la liste des dimanches de cette année.

In [ ]:
patron = """
<ul>{% for i, url in enumerate(urls) %}
  <li><a href="{{ url }}">url {{ i }}</a></li>{% endfor %}
</ul>
"""

In [ ]:
from jinja2 import Template

tpl = Template(patron)

In [ ]:
print(tpl.render(urls=urls[:10], enumerate=enumerate))